# RAG 시스템 - 통합 실행 노트북

이 노트북은 RAG 시스템의 모든 기능을 한 번에 실행할 수 있습니다:
1. 환경 설정 확인
2. 문서 저장
3. 문서 조회
4. 질의-응답 생성

## 1. 필요한 라이브러리 임포트 및 환경 설정

In [ ]:
import json
import requests
from typing import List, Dict, Any

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 서버 설정
CHROMA_HOST = "localhost"
CHROMA_PORT = "8090"
OLLAMA_HOST = "localhost"
OLLAMA_PORT = "11434"

# ChromaDB 설정
CHROMA_SETTINGS = {
    "chroma_db_impl": "duckdb+parquet",
    "persist_directory": "chroma",
    "anonymized_telemetry": False
}

## 2. 서버 상태 확인

In [ ]:
def check_servers():
    """ChromaDB와 Ollama 서버 상태 확인"""
    servers = {
        'ChromaDB': f'http://{CHROMA_HOST}:{CHROMA_PORT}/api/v1/heartbeat',
        'Ollama': f'http://{OLLAMA_HOST}:{OLLAMA_PORT}/api/tags'
    }
    
    for name, url in servers.items():
        try:
            response = requests.get(url)
            if response.status_code == 200:
                print(f"✅ {name} 서버가 정상적으로 실행 중입니다.")
            else:
                print(f"❌ {name} 서버에 연결할 수 없습니다. 상태 코드: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"❌ {name} 서버에 연결할 수 없습니다. 오류: {str(e)}")

# 서버 상태 확인
check_servers()

## 3. 임베딩 모델 설정

In [ ]:
def get_embeddings():
    """한국어에 최적화된 임베딩 모델 생성"""
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sbert-nli",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    return embeddings

## 4. 문서 저장 함수

In [ ]:
def store_documents_in_chroma(documents: List[Dict[str, Any]], collection_name: str = "rag_test"):
    """문서를 벡터로 변환하여 ChromaDB에 저장"""
    # 텍스트 분할 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", ".", "!", "?", "。", "！", "？", " ", ""]
    )
    
    # 문서 처리
    texts = []
    metadatas = []
    
    for doc in documents:
        chunks = text_splitter.split_text(doc['content'])
        texts.extend(chunks)
        
        # 각 청크에 대한 메타데이터 생성
        for _ in chunks:
            metadata = {
                'source': doc.get('source', 'unknown'),
                'author': doc.get('author', 'unknown'),
                'date': doc.get('date', 'unknown')
            }
            metadatas.append(metadata)
    
    # ChromaDB에 저장
    embeddings = get_embeddings()
    vectordb = Chroma(
        collection_name=collection_name,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
    )
    
    vectordb.add_texts(
        texts=texts,
        metadatas=metadatas
    )
    
    print(f"✅ {len(texts)}개의 청크가 ChromaDB에 저장되었습니다.")
    return vectordb

## 5. 문서 조회 함수

In [ ]:
def load_vector_store(collection_name: str = "rag_test"):
    """ChromaDB에서 벡터 스토어 로드"""
    try:
        embeddings = get_embeddings()
        vector_store = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            client_settings=CHROMA_SETTINGS
        )
        print(f"✅ 컬렉션 '{collection_name}'을 성공적으로 로드했습니다.")
        return vector_store
    except Exception as e:
        print(f"❌ 벡터 스토어 로드 중 오류 발생: {str(e)}")
        return None

## 6. LLM 질의-응답 함수

In [ ]:
def query_ollama(prompt: str, context: str = "") -> str:
    """Ollama API를 사용하여 LLM 모델 호출"""
    url = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}/api/generate"
    
    system_prompt = """
    너는 주어진 컨텍스트를 기반으로 질문에 답변하는 도우미야.
    다음 규칙을 반드시 따라야 해:
    1. 컨텍스트에 있는 정보만 사용해서 답변해야 해
    2. 컨텍스트에 없는 내용은 '주어진 정보에는 없습니다'라고 말해야 해
    3. 답변은 한국어로 해야 해
    4. 답변은 친절하고 자연스러운 어투를 사용해야 해
    """
    
    if context:
        full_prompt = f"{system_prompt}\n\n컨텍스트:\n{context}\n\n질문: {prompt}"
    else:
        full_prompt = f"{system_prompt}\n\n질문: {prompt}"
    
    try:
        response = requests.post(url, json={
            "model": "deepseek-r1:8b",
            "prompt": full_prompt,
            "stream": False
        })
        
        if response.status_code == 200:
            return response.json()["response"]
        else:
            return f"❌ API 호출 실패: {response.status_code}"
            
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"

In [ ]:
def run_rag_query(query: str, collection_name: str = "rag_test", k: int = 3):
    """벡터DB에서 질문에 대한 답변을 검색"""
    # 벡터 스토어 로드
    vector_store = load_vector_store(collection_name)
    if not vector_store:
        return "❌ 벡터 스토어를 로드할 수 없습니다."
    
    try:
        # 유사도 검색
        results = vector_store.similarity_search_with_relevance_scores(query, k=k)
        
        if not results:
            return "관련된 문서를 찾을 수 없습니다."
        
        # 컨텍스트 구성
        contexts = []
        for doc, score in results:
            if score < 0.5:  # 유사도가 낮은 문서는 제외
                continue
            contexts.append(f"[유사도: {score:.2f}] {doc.page_content}")
        
        context = "\n\n".join(contexts)
        
        # LLM으로 답변 생성
        answer = query_ollama(query, context)
        
        return {
            "query": query,
            "context": context,
            "answer": answer
        }
        
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"

## 7. 예제 실행

### 7.1 샘플 문서 저장

In [ ]:
# 예제 문서
sample_documents = [
    {
        "content": """
        한국의 봄은 3월부터 5월까지로, 꽃이 피고 날씨가 따뜻해지는 계절입니다.
        특히 벚꽃이 피는 4월은 많은 사람들이 꽃구경을 가는 시기입니다.
        봄에는 황사와 미세먼지가 있을 수 있지만, 새싹이 돋고 생명이 움트는 아름다운 계절입니다.
        """,
        "source": "계절 설명서",
        "author": "김계절",
        "date": "2024-03-01"
    },
    {
        "content": """
        한국의 여름은 6월부터 8월까지입니다. 높은 기온과 습도가 특징이며,
        장마철에는 집중적인 강우가 있습니다. 에어컨 사용이 늘어나고,
        시원한 음식을 즐기는 계절입니다. 해수욕장이나 계곡으로 피서를 가는 것이 일반적입니다.
        """,
        "source": "계절 설명서",
        "author": "김계절",
        "date": "2024-03-01"
    }
]

# 문서 저장
vectordb = store_documents_in_chroma(sample_documents)

### 7.2 문서 조회

In [ ]:
# 벡터 스토어 로드
vector_store = load_vector_store()

# 저장된 문서 수 확인
if vector_store:
    print(f"저장된 문서 수: {vector_store._collection.count()}")

### 7.3 질문하기

In [ ]:
# 예제 질문
questions = [
    "한국의 봄은 어떤 특징이 있나요?",
    "한국의 여름철 날씨는 어떤가요?",
    "한국의 가을은 어떤 특징이 있나요?"  # 데이터에 없는 내용
]

for question in questions:
    print(f"\n질문: {question}")
    result = run_rag_query(question)
    if isinstance(result, dict):
        print("\n답변:")
        print(result["answer"])
        print("\n참고한 컨텍스트:")
        print(result["context"])
    else:
        print(result)